## 0. Setup
The following is the setup code for pytorch3d.

In [ ]:
import os
import sys
import torch
need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True
if need_pytorch3d:
    if torch.__version__.startswith(("1.13.", "2.0.")) and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install fvcore iopath
        !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=9841237ed1f9311d3444a8fb83bdfa2fe2e7aeea4369708953cbab3d1fa5d4d7
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31530 sha256=c833915d31f36b279e04b1513225911ee4eb7ad6631a8a3aae03ae80f883dacd
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath
Looking in links: https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/py310_cu118_pyt201/download.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 119.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


Change the package directory to your own to recreate the results

In [ ]:
import os
package_directory = '/content/drive/MyDrive/CMSC848F_Assignment1/assignment1-main'
os.chdir(package_directory)


In [ ]:
import torch
import pytorch3d
from pytorch3d.structures import Meshes
from pytorch3d.renderer import FoVPerspectiveCameras, PointLights
from pytorch3d.renderer import look_at_view_transform

In [ ]:
import imageio
import numpy as np
from PIL import Image, ImageDraw
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from starter.utils import get_mesh_renderer,get_device,load_cow_mesh

### 0.1 Rendering your first mesh

In [ ]:
def render_cow(cow_path="data/cow.obj", image_size=256, color=[0.2, 0.4, 0.5], device=None,):

    if device is None:
        device = get_device()

    # Get the renderer.
    renderer = get_mesh_renderer(image_size=image_size)

    # Get the vertices, faces, and textures.
    vertices, faces = load_cow_mesh(cow_path)
    vertices = vertices.unsqueeze(0)  # (N_v, 3) -> (1, N_v, 3)
    faces = faces.unsqueeze(0)  # (N_f, 3) -> (1, N_f, 3)
    textures = torch.ones_like(vertices)  # (1, N_v, 3)
    textures = textures * torch.tensor(color)  # (1, N_v, 3)
    mesh = pytorch3d.structures.Meshes(
        verts=vertices,
        faces=faces,
        textures=pytorch3d.renderer.TexturesVertex(textures),
    )
    mesh = mesh.to(device)

    # Prepare the camera:
    cameras = pytorch3d.renderer.FoVPerspectiveCameras(
        R=torch.eye(3).unsqueeze(0), T=torch.tensor([[0, 0, 3]]), fov=60, device=device
    )

    # Place a point light in front of the cow.
    lights = pytorch3d.renderer.PointLights(location=[[0, 0, -3]], device=device)

    rend = renderer(mesh, cameras=cameras, lights=lights)
    rend = rend.cpu().numpy()[0, ..., :3]  # (B, H, W, 4) -> (H, W, 3)
    # The .cpu moves the tensor to GPU (if needed).
    return rend

change the output path to your own

In [ ]:
cow_path = "data/cow.obj"
image_size = 256
device = "cpu"
output_path = "output_images_and_GIFS/cow_render_new.jpg"
image = render_cow(cow_path=cow_path, image_size=image_size, device = device)
plt.imsave(output_path, image)



## 1. Practicing with Cameras

## 1.1. 360-degree Renders


In [ ]:
def render_cow(cow_path="data/cow.obj",
               image_size=256,
               color=[0.3, 0.5, 0.5],
               device=None):

    # Step 1: Create the Renderer
    renderer = get_mesh_renderer(image_size=image_size)

    # Step 2: Load Cow Mesh Data
    vertices, faces = load_cow_mesh(cow_path)

    # Step 3: Prepare Textures
    # Create identity texture (same size as faces) and apply color
    textures = torch.ones_like(vertices) * torch.tensor(color, dtype=torch.float32)

    # Convert vertices, faces, and textures to tensors
    vertices = vertices.unsqueeze(0)
    faces = faces.unsqueeze(0)
    textures = textures.unsqueeze(0)

    theta_degrees=10
    angles = np.linspace(1, 360, theta_degrees, endpoint=False)
    renders=[]

    for angle in angles:

      # Create a new mesh for each viewpoint
      mesh = pytorch3d.structures.Meshes(
          verts=vertices,
          faces=faces,
          textures=pytorch3d.renderer.TexturesVertex(textures),
      )
      mesh = mesh.to(device)

      # Prepare the camera transformation for the current angle
      R, T = pytorch3d.renderer.cameras.look_at_view_transform(dist=5.0, elev=0, azim=angle, degrees=True, eye=None, at=((0, 0, 0),), up=((0, 1, 0),), device=device)
      cameras = pytorch3d.renderer.FoVPerspectiveCameras(
          R=R, T=T, fov=60, device=device
      )
      lights = pytorch3d.renderer.PointLights(location=[[0, 0, -3]], device=device)
      # Render the image for the current viewpoint
      rend = renderer(mesh, cameras=cameras, lights=lights)
      rend=rend.cpu().numpy()[0, ..., :3]
      rend = (rend * 255)
      renders.append(rend)

    rendered_image = np.array(renders, dtype=np.uint8)
    return rendered_image


change the output path in the imageio.mimsave function to your own to store the GIF

In [ ]:
cow_path = "data/cow.obj"
image_size = 256
device = "cpu"
image = render_cow(cow_path=cow_path, image_size=image_size, device = device)
imageio.mimsave('output_images_and_GIFS/360_Cow.gif',image, duration=10,loop=0)

In [ ]:
import imageio
import numpy as np
from PIL import Image, ImageDraw
from tqdm.auto import tqdm

### 1.2 Re-creating the Dolly Zoom

In [ ]:
def dolly_zoom(image_size=256,
    num_frames=10,
    duration=3,
    device=None,
    output_file="output/dolly_new.gif"):

  # Check if a device is specified, otherwise get the default device
  if device is None:
      device = get_device()

  # Load a 3D mesh from an OBJ file and move it to the specified device
  mesh = pytorch3d.io.load_objs_as_meshes(["data/cow_on_plane.obj"])
  mesh = mesh.to(device)

  # Create a mesh renderer with the specified image size and device
  renderer = get_mesh_renderer(image_size=image_size, device=device)

  # Define a point light source
  lights = pytorch3d.renderer.PointLights(location=[[0.0, 0.0, -3.0]], device=device)

  # Generate a sequence of field of view (FOV) angles for zooming
  fovs = torch.linspace(5, 120, num_frames)
  fov1=torch.deg2rad(torch.tensor(5,dtype=torch.float32,device=device))
  d1=50
  renders = []

  # Loop through FOV angles and render frames
  for fov in tqdm(fovs):
    fov2=torch.deg2rad(fov)
    distance = d1 * (torch.tan(fov1 / 2) / torch.tan(fov2 / 2))
    T = torch.tensor([[ 0,0, distance]],dtype=torch.float32,device=device)

    # Create a perspective camera with the current FOV and distance
    cameras = pytorch3d.renderer.FoVPerspectiveCameras(fov=fov, T=T, device=device)

    # Render the mesh with the current camera and lighting
    rend = renderer(mesh, cameras=cameras, lights=lights)
    rend = rend[0, ..., :3].cpu().numpy()  # (N, H, W, 3)
    renders.append(rend)
    fov1=fov2
    d1=distance

  images = []

  # Loop through rendered frames and add FOV labels
  for i, r in enumerate(renders):
      image = Image.fromarray((r * 255).astype(np.uint8))
      draw = ImageDraw.Draw(image)
      # Add the FOV label to the image
      draw.text((20, 20), f"fov: {fovs[i]:.2f}", fill=(255, 100, 100))
      images.append(np.array(image))

  # Calculate the frame duration based on duration and num_frames
  frame_duration = int((duration * 1000) / num_frames)
  imageio.mimsave(output_file, images, duration=frame_duration)



In [ ]:
num_frames=10
duration=3
output_file="output_images_and_GIFS/dolly_new.gif"
image_size=256
dolly_zoom(
  image_size=image_size,
  num_frames=num_frames,
  duration=duration,
  output_file=output_file,
)

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
from starter.utils import get_device, get_mesh_renderer

## 2. Practicing with Meshes

### 2.1 Constructing a Tetrahedron

In [ ]:
def create_tetrahedron_mesh():
    # Define the vertices of the tetrahedron
    vertices = torch.tensor([
        [0.0, 0.0, 0.0],
        [1.0, 0.0, 0.0],
        [0.5, np.sqrt(3) / 2.0, 0.0],
        [0.5, np.sqrt(3) / 6.0, np.sqrt(2) / np.sqrt(3)],
    ], dtype=torch.float32)

    # Define the faces (vertex indices of triangles)
    faces = torch.tensor([
        [0, 1, 2],
        [0, 1, 3],
        [1, 2, 3],
        [0, 2, 3],
    ], dtype=torch.int64)

    # Create a single-color texture (e.g., blue)
    color = [0.0, 0.0, 1.0]  # Blue
    textures = torch.ones_like(vertices).unsqueeze(0)
    textures = textures * torch.tensor(color, dtype=torch.float32)

    return vertices, faces, textures


In [ ]:
from pytorch3d.renderer import TexturesVertex
from pytorch3d.renderer.cameras import PerspectiveCameras

def create_tetrahedron_gif(cow_path="data/cow.obj", image_size=256, theta_degrees=36):
    # Get the computing device (e.g., GPU or CPU)
    device = get_device()

    # Initialize the mesh renderer with the specified image size and device
    renderer = get_mesh_renderer(image_size=image_size, device=device)

    # Create a point light source
    lights = PointLights(location=[[0.0, 0.0, -3.0]], device=device)

    # Create a tetrahedron mesh from the function created above
    vertices, faces, textures = create_tetrahedron_mesh()

    # Generate a sequence of rotation angles (thetas) for animation
    thetas = np.linspace(0, 360, theta_degrees, endpoint=False)
    renders = []

    # Loop through rotation angles and render frames
    for theta in tqdm(thetas):
        # Create a mesh using the current vertices, faces, and textures
        mesh = Meshes(
            verts=vertices.unsqueeze(0).to(device),
            faces=faces.unsqueeze(0).to(device),
            textures=TexturesVertex(textures).to(device),
        )

        # Define the rotation and translation transformations for the camera
        R, T = pytorch3d.renderer.cameras.look_at_view_transform(
            dist=2.0, elev=0, azim=theta, degrees=True, eye=None, at=((0, 0, 0),), up=((0, 1, 0),), device=device
        )

        # Create a perspective camera with the specified transformation and FOV
        cameras = FoVPerspectiveCameras(
            R=R, T=T, fov=90, device=device,
        )

        # Render the mesh with the current camera and lighting
        rend = renderer(meshes_world=mesh, cameras=cameras, lights=lights)

        # Extract the RGB channels and adjust the intensity range
        rend = rend[0, ..., :3].cpu().numpy()  # (N, H, W, 3)
        rend = (rend * 255)

        # Append the rendered frame to the list
        renders.append(rend)

    # Convert the list of rendered frames to an array of images
    rendered_images = np.array(renders, dtype=np.uint8)

    return rendered_images


In [ ]:
image_size = 256
image = create_tetrahedron_gif(image_size=image_size)
imageio.mimsave("output_images_and_GIFS/tetrahedron.gif",image, duration=10,loop=0)

  0%|          | 0/36 [00:00<?, ?it/s]

### 2.2 Constructing a Cube

In [ ]:
def create_cube_mesh():

  # Define the vertices and faces for a cube
    vertices = torch.tensor([
    [0, 0, 0],  # Vertex 0
    [0.5, 0, 0],  # Vertex 1
    [0.5, 0.5, 0],  # Vertex 2
    [0, 0.5, 0],  # Vertex 3
    [0, 0, 0.5],  # Vertex 4
    [0.5, 0, 0.5],  # Vertex 5
    [0.5, 0.5, 0.5],  # Vertex 6
    [0, 0.5, 0.5],  # Vertex 7
  ], dtype=torch.float32)

  # Define the triangular faces of the cube
    faces = torch.tensor([
    [0, 3, 2], [0, 2, 1],  # Bottom face
    [4, 7, 6], [4, 6, 5],  # Top face
    [0, 4, 5], [0, 5, 1],  # Front face
    [1, 2, 6], [1, 6, 5],  # Right face
    [2, 3, 7], [2, 7, 6],  # Back face
    [0, 3, 7], [0, 7, 4],  # Left face
  ], dtype=torch.int64)

    # Create a single-color texture (e.g., blue)
    color = [0.0, 0.0, 1.0]  # Blue
    textures = torch.ones_like(vertices).unsqueeze(0)
    textures = textures * torch.tensor(color, dtype=torch.float32)

    return vertices, faces, textures

In [ ]:
def create_cube_gif(cow_path="data/cow.obj", image_size=256, theta_degrees=36):
    device = get_device()
    renderer = get_mesh_renderer(image_size=image_size, device=device)
    lights = PointLights(location=[[0.0, 0.0, -3.0]], device=device)
    # Create the cube mesh
    vertices, faces, textures = create_cube_mesh()
    # Define rotation angles
    thetas = np.linspace(0, 360, theta_degrees, endpoint=False)
    renders = []
    for theta in tqdm(thetas):
        # Create a Meshes object with updated rotation
        mesh = Meshes(
            verts=[vertices.to(device)],
            faces=[faces.to(device)],
            textures=TexturesVertex(textures).to(device),
        )
        mesh = mesh.to(device)

        R, T = look_at_view_transform(dist=2.0, elev=0, azim=theta, degrees=True, eye=None, at=((0, 0, 0),), up=((0, 1, 0),), device=device)
        cameras = FoVPerspectiveCameras(
            R=R, T=T, fov=90, device=device
        )

        # Render the image for the current viewpoint
        rend = renderer(meshes_world=mesh, cameras=cameras, lights=lights)
        rend = rend[0, ..., :3].cpu().numpy()
        rend = (rend * 255).astype(np.uint8)  # (H, W, 3)
        renders.append(rend)
    rendered_images = np.array(renders, dtype=np.uint8)
    return rendered_images

In [ ]:
image_size = 256
image = create_cube_gif(image_size=image_size)
imageio.mimsave("output_images_and_GIFS/cube.gif",image, duration=10,loop=0)

  0%|          | 0/36 [00:00<?, ?it/s]

## 3. Re-texturing a mesh

In [ ]:
def retexture_cow_mesh(cow_path="data/cow.obj", image_size=256, theta_degrees=36):
    device = get_device()
    # Get the mesh renderer.
    renderer = get_mesh_renderer(image_size=image_size)
    # Create point lights for rendering
    lights = pytorch3d.renderer.PointLights(location=[[0, 0, -3]], device=device)

    # Define two colors for texture blending
    color1 = [0.6, 0.9, 0.1]
    color2 = [0.2, 0.4, 0.0]
    # Load the cow mesh vertices and faces
    vertices, faces = load_cow_mesh(cow_path)

    # Calculate the minimum and maximum Z-coordinate values
    z_min = vertices[:, 2].min()
    z_max = vertices[:, 2].max()

    # Compute alpha values for each vertex based on their Z-coordinates
    alpha = (vertices[:, 2] - z_min) / (z_max - z_min)
    alpha = alpha.unsqueeze(-1)  # Reshape alpha to (N_v, 1)

    # Interpolate colors based on alpha values
    color1 = torch.tensor(color1)
    color2 = torch.tensor(color2)
    color = alpha * color2 + (1 - alpha) * color1

    # Create textures using the color information
    textures = color.unsqueeze(0)  # Reshape color to (1, N_v, 3)
    vertices = vertices.unsqueeze(0)
    faces = faces.unsqueeze(0)

    # Generate a sequence of rotation angles (thetas) for animation
    thetas = np.linspace(1, 360, theta_degrees, endpoint=False)
    renders = []

    # Loop through rotation angles and render frames
    for theta in tqdm(thetas):

        # Create a new mesh for each viewpoint
        mesh = pytorch3d.structures.Meshes(
            verts=vertices,
            faces=faces,
            textures=pytorch3d.renderer.TexturesVertex(textures),
        )
        mesh = mesh.to(device)

        # Prepare the camera transformation for the current angle
        R, T = pytorch3d.renderer.cameras.look_at_view_transform(
            dist=2.5, elev=0, azim=theta, degrees=True, eye=None, at=((0, 0, 0),), up=((0, 1, 0),), device=device
        )
        cameras = pytorch3d.renderer.FoVPerspectiveCameras(
            R=R, T=T, fov=90, device=device
        )

        # Render the image for the current viewpoint
        rend = renderer(mesh, cameras=cameras, lights=lights)
        rend = rend.cpu().numpy()[0, ..., :3]
        rend = (rend * 255)
        renders.append(rend)

    # Convert the list of rendered frames to an array of images
    rendered_images = np.array(renders, dtype=np.uint8)

    return rendered_images


In [ ]:
image_size = 256
image = retexture_cow_mesh(image_size=image_size)
imageio.mimsave("output_images_and_GIFS/retextured_cow.gif",image, duration=10,loop=0)

  0%|          | 0/36 [00:00<?, ?it/s]

## 4. Camera Transformations

In [ ]:
def camera_transformations(cow_path="data/cow_with_axis.obj", image_size=256,
                          R_relative=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
                          T_relative=[0, 0, 0],
                          device=None):
    # Check if a device is specified, otherwise get the default device
    if device is None:
        device = get_device()

    # Load the cow mesh using PyTorch3D
    meshes = pytorch3d.io.load_objs_as_meshes([cow_path], device=device)

    # Convert the relative rotation and translation to tensors
    R_relative = torch.tensor(R_relative).float()
    T_relative = torch.tensor(T_relative).float()

    # Calculate the final rotation and translation transformations
    R = R_relative @ torch.tensor([[1.0, 0, 0], [0, 1, 0], [0, 0, 1]])
    T = R_relative @ torch.tensor([0.0, 0, 3]) + T_relative

    # Get the mesh renderer
    renderer = get_mesh_renderer(image_size=image_size)

    # Define the camera with combined transformations
    cameras = FoVPerspectiveCameras(
        R=R.t().unsqueeze(0),  # Transpose for PyTorch3D's convention
        T=T.unsqueeze(0),
        fov=90,  # Adjust the field of view as needed
        device=device
    )

    # Define lights for rendering
    lights = PointLights(location=[[0, 0.0, -3.0]], device=device)

    # Render the image
    rend = renderer(meshes, cameras=cameras, lights=lights)

    # Extract the RGB channels from the rendered image
    rend = rend[0, ..., :3].cpu().numpy()

    return rend


#### Zeroth Transform (T_0, R_0)

In [ ]:
image_size = 256
cow_path = "data/cow_with_axis.obj"
image = camera_transformations(cow_path=cow_path, image_size=image_size)
plt.imsave("output_images_and_GIFS/zeroth_transform_cow.jpg", image)

#### First Transform (T_1, R_1)

In [ ]:
image_size = 256
cow_path = "data/cow_with_axis.obj"
image = camera_transformations(cow_path=cow_path, image_size=image_size,  R_relative=[[0, -1, 0], [1, 0, 0], [0, 0, 1]],
                              T_relative=[0, 0, 2])
plt.imsave("output_images_and_GIFS/first_transform_cow.jpg", image)

#### Second Transform (T_2, R_2)

In [ ]:
image_size = 256
cow_path = "data/cow_with_axis.obj"
image = camera_transformations(cow_path=cow_path, image_size=image_size,R_relative=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
                              T_relative=[0, 0, 2])
plt.imsave("output_images_and_GIFS/second_transform_cow.jpg", image)

#### Third Transform (T_3, R_3)

In [ ]:
image_size = 256
cow_path = "data/cow_with_axis.obj"
image = camera_transformations(cow_path=cow_path, image_size=image_size,R_relative=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
                               T_relative=[0.52, -0.5, 0])
plt.imsave("output_images_and_GIFS/third_transform_cow.jpg", image)

#### Fourth Transform (T_4, R_4)

In [ ]:
image_size = 256
cow_path = "data/cow_with_axis.obj"
image = camera_transformations(cow_path=cow_path, image_size=image_size,R_relative=[[0, 0, -1], [0, 1, 0], [1, 0, 0]],
                               T_relative=[3, 0, 3])
plt.imsave("output_images_and_GIFS/fourth_transform_cow.jpg", image)

## 5. Rendering Generic 3D Representations

### 5.1 Rendering Point Clouds from RGB-D Images

In [ ]:
pip install PyMCubes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.3/274.3 kB 5.2 MB/s eta 0:00:00


In [ ]:
import pickle
import matplotlib.pyplot as plt
import mcubes
import numpy as np
import pytorch3d
import torch
import imageio
from starter.utils import get_device, get_mesh_renderer, get_points_renderer, unproject_depth_image


def load_rgbd_data(path="data/rgbd_data.pkl"):
    with open(path, "rb") as f:
        data = pickle.load(f)
    return data

In [ ]:
def render_bridge(point_cloud_path="data/bridge_pointcloud.npz", image_size=256, background_color=(1, 1, 1), device=None):
    """
    Renders a point cloud.
    """
    if device is None:
        device = get_device()

    # Get the point cloud renderer
    renderer = get_points_renderer(
        image_size=image_size, background_color=background_color
    )

    # Load RGB-D data from a source (data should include RGB images, masks, depths, and camera parameters)
    data = load_rgbd_data()

    # Extract RGB image, mask, depth, and camera parameters for the first viewpoint
    image = torch.tensor(data.get("rgb1"))
    mask = torch.tensor(data.get("mask1"))
    depth = torch.tensor(data.get("depth1"))
    camera = data.get("cameras1")

    # Unproject the depth image to obtain 3D points and their RGB features
    points, rgb = unproject_depth_image(image, mask, depth, camera)

    # Prepare the vertices and RGB features for rendering
    verts = points.unsqueeze(0)
    rgb = rgb.unsqueeze(0)

    # Define the number of rotation angles (theta_degrees) for animation
    theta_degrees = 5
    thetas = np.linspace(1, 360, theta_degrees, endpoint=False)

    # Initialize a list to store rendered frames for the first viewpoint
    my_images1 = []

    # Loop through rotation angles and render frames for the first viewpoint
    for theta in tqdm(thetas):
        # Create a point cloud structure with the 3D points and RGB features
        point_cloud = pytorch3d.structures.Pointclouds(points=verts, features=rgb)

        # Define the camera transformation for the current angle
        R, T = pytorch3d.renderer.look_at_view_transform(6, 0, theta, up=((0, -1, 0),))
        cameras = pytorch3d.renderer.FoVPerspectiveCameras(R=R, T=T)

        # Render the image for the current viewpoint
        rend = renderer(point_cloud, cameras=cameras)
        rend = rend.cpu().numpy()[0, ..., :3]  # (B, H, W, 4) -> (H, W, 3)
        rend = (rend * 255)
        my_images1.append(rend)

    rendered_image_1 = np.array(my_images1, dtype=np.uint8)

    # Repeat the above process for the second viewpoint (rgb2)
    image = torch.tensor(data.get("rgb2"))
    mask = torch.tensor(data.get("mask2"))
    depth = torch.tensor(data.get("depth2"))
    camera = data.get("cameras2")
    points1, rgb1 = unproject_depth_image(image, mask, depth, camera)
    verts1 = points1.unsqueeze(0)
    rgb1 = rgb1.unsqueeze(0)
    my_images2 = []
    for theta in tqdm(thetas):
        point_cloud = pytorch3d.structures.Pointclouds(points=verts1, features=rgb1)
        R, T = pytorch3d.renderer.look_at_view_transform(6, 0, theta, up=((0, -1, 0),))
        cameras = pytorch3d.renderer.FoVPerspectiveCameras(R=R, T=T)
        rend = renderer(point_cloud, cameras=cameras)
        rend = rend.cpu().numpy()[0, ..., :3]  # (B, H, W, 4) -> (H, W, 3)
        rend = (rend * 255)
        my_images2.append(rend)
    rendered_image_2 = np.array(my_images2, dtype=np.uint8)

    # Concatenate points and RGB features from both viewpoints
    points2 = torch.cat((points.unsqueeze(0), points1.unsqueeze(0)), dim=1).squeeze(0)
    rgb2 = torch.cat((rgb, rgb1), dim=1).squeeze(0)
    verts2 = points2.unsqueeze(0)
    rgb2 = rgb2.unsqueeze(0)

    # Initialize a list to store rendered frames for the combined viewpoint
    my_images3 = []

    # Loop through rotation angles and render frames for the combined viewpoint
    for theta in tqdm(thetas):
        point_cloud = pytorch3d.structures.Pointclouds(points=verts2, features=rgb2)
        R, T = pytorch3d.renderer.look_at_view_transform(6, 0, theta, up=((0, -1, 0),))
        cameras = pytorch3d.renderer.FoVPerspectiveCameras(R=R, T=T)
        rend = renderer(point_cloud, cameras=cameras)
        rend = rend.cpu().numpy()[0, ..., :3]  # (B, H, W, 4) -> (H, W, 3)
        rend = (rend * 255)
        my_images3.append(rend)

    # Convert the list of rendered frames to an array of images for the combined viewpoint
    rendered_image_3 = np.array(my_images3, dtype=np.uint8)

    # Return the rendered images for all three viewpoints
    return rendered_image_1, rendered_image_2, rendered_image_3


### 5.2 Parametric Functions

In [ ]:
def render_torus(image_size=256, num_samples=250, device=None):
    """
    Renders a torus using parametric sampling. Samples num_samples ** 2 points.
    """

    # Check if a device is specified, otherwise get the default device
    if device is None:
        device = get_device()

    # Generate a grid of phi and theta values
    phi = torch.linspace(0, 2 * np.pi, num_samples)
    theta = torch.linspace(0, 2*np.pi, num_samples)

    # Densely sample phi and theta on a grid
    Phi, Theta = torch.meshgrid(phi, theta, indexing='xy')

    # Define the radii for the torus
    R = torch.tensor(3)
    r = torch.tensor(2)

    # Calculate the x, y, and z coordinates of points on the torus
    x = (R + r * torch.cos(Theta)) * torch.cos(Phi)
    y = (R + r * torch.cos(Theta)) * torch.sin(Phi)
    z = r * torch.sin(Theta)

    # Stack the coordinates to create a point cloud
    points = torch.stack((x.flatten(), y.flatten(), z.flatten()), dim=1)

    # Normalize the colors based on the point cloud coordinates
    color = (points - points.min()) / (points.max() - points.min())

    # Create a point cloud object with the points and colors
    torus_point_cloud = pytorch3d.structures.Pointclouds(
        points=[points], features=[color],
    ).to(device)

    # Define the number of rotation angles (theta_degrees) for animation
    theta_degrees = 90
    thetas = np.linspace(1, 360, theta_degrees, endpoint=False)
    renders = []

    # Loop through rotation angles and render frames
    for theta in tqdm(thetas):
        # Define the camera transformation for the current angle
        R, T = pytorch3d.renderer.look_at_view_transform(10, 25, theta)
        cameras = pytorch3d.renderer.FoVPerspectiveCameras(R=R, T=T, device=device)

        # Get the point cloud renderer
        renderer = get_points_renderer(image_size=image_size, device=device)

        # Render the image for the current viewpoint
        rend = renderer(torus_point_cloud, cameras=cameras)
        rend = rend.cpu().numpy()[0, ..., :3]
        rend = (rend * 255)
        renders.append(rend)

    rendered_image = np.array(renders, dtype=np.uint8)

    return rendered_image


### 5.3 Implicit Surfaces

In [ ]:
def render_torus_mesh(image_size=256, voxel_size=64, device=None):
    """
    Renders a torus mesh using Marching Cubes algorithm.
    """
    if device is None:
        device = get_device()

    # Define the range of coordinates in the voxel grid
    min_value = -5.6
    max_value = 5.6

    # Create a 3D voxel grid
    X, Y, Z = torch.meshgrid([torch.linspace(min_value, max_value, voxel_size)] * 3)

    # Define radii for the torus
    R = torch.tensor(3)
    r = torch.tensor(2)

    # Compute voxel values using the torus equation
    voxels = (X**2 + Y**2 + Z**2 + R**2 - r**2)**2 - torch.tensor(4) * (R**2) * (X**2 + Y**2)

    # Extract vertices and faces using Marching Cubes
    vertices, faces = mcubes.marching_cubes(mcubes.smooth(voxels), isovalue=0)
    vertices = torch.tensor(vertices).float()
    faces = torch.tensor(faces.astype(int))

    # Normalize vertex coordinates to the original coordinate system
    vertices = (vertices / voxel_size) * (max_value - min_value) + min_value

    # Calculate vertex textures based on normalized vertex coordinates
    textures = (vertices - vertices.min()) / (vertices.max() - vertices.min())
    textures = pytorch3d.renderer.TexturesVertex(vertices.unsqueeze(0))

    # Create a torus mesh
    mesh = pytorch3d.structures.Meshes([vertices], [faces], textures=textures).to(device)

    # Define point lights for rendering
    lights = pytorch3d.renderer.PointLights(location=[[0, 0.0, -4.0]], device=device)

    # Get the mesh renderer
    renderer = get_mesh_renderer(image_size=image_size, device=device)

    # Define the number of rotation angles (theta_degrees) for animation
    theta_degrees = 36
    thetas = np.linspace(1, 360, theta_degrees, endpoint=False)
    renders = []

    # Loop through rotation angles and render frames
    for theta in tqdm(thetas):

        # Define the camera transformation for the current angle
        R, T = pytorch3d.renderer.look_at_view_transform(dist=10, elev=0, azim=theta)
        cameras = pytorch3d.renderer.FoVPerspectiveCameras(R=R, T=T, device=device)

        # Render the image for the current viewpoint
        rend = renderer(mesh, cameras=cameras, lights=lights)
        rend = rend[0, ..., :3].detach().cpu().numpy().clip(0, 1)
        rend = (rend * 255)
        renders.append(rend)

    rendered_image = np.array(renders, dtype=np.uint8)

    return rendered_image


In [ ]:
image_size = 256
render = "point_cloud"
image1,image2,image3 = render_bridge(image_size=image_size)
imageio.mimsave('output_images_and_GIFS/plant1.gif',image1, duration=100,loop=0)
imageio.mimsave('output_images_and_GIFS/plant2.gif',image2, duration=100,loop=0)
imageio.mimsave('output_images_and_GIFS/plant3.gif',image3, duration=100,loop=0)

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
image_size = 256
render = "parametric"
num_samples = 100
image = render_torus(image_size=image_size, num_samples=num_samples)
imageio.mimsave('output_images_and_GIFS/torus.gif',image, duration=100,loop=0)

  0%|          | 0/90 [00:00<?, ?it/s]

In [ ]:
image_size = 256
render = "implicit"
image = render_torus_mesh(image_size=image_size)
imageio.mimsave('output_images_and_GIFS/implicit_torus.gif',image, duration=100,loop=0)

  0%|          | 0/36 [00:00<?, ?it/s]

In [ ]:
image_size = 256
render = "parametric"
num_samples = 200
image = render_torus(image_size=image_size, num_samples=num_samples)
imageio.mimsave('output_images_and_GIFS/torus_200.gif',image, duration=100,loop=0)